# Ungraded Lab: Build a Multi-output Model

In this lab, we'll show how you can build models with more than one output. The dataset we will be working on is available from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Energy+efficiency). It is an Energy Efficiency dataset which uses the bulding features (e.g. wall area, roof area) as inputs and has two outputs: Cooling Load and Heating Load. Let's see how we can build a model to train on this data.

## Imports

In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
from sklearn.model_selection import train_test_split

## Utilities

We define a few utilities for data conversion and visualization to make our code more neat.

In [2]:
def format_output(data):
    y1 = data.pop('Y1')
    y1 = np.array(y1)
    y2 = data.pop('Y2')
    y2 = np.array(y2)
    return y1, y2


def norm(x):
    return (x - train_stats['mean']) / train_stats['std']


def plot_diff(y_true, y_pred, title=''):
    plt.scatter(y_true, y_pred)
    plt.title(title)
    plt.xlabel('True Values')
    plt.ylabel('Predictions')
    plt.axis('equal')
    plt.axis('square')
    plt.xlim(plt.xlim())
    plt.ylim(plt.ylim())
    plt.plot([-100, 100], [-100, 100])
    plt.show()


def plot_metrics(metric_name, title, ylim=5):
    plt.title(title)
    plt.ylim(0, ylim)
    plt.plot(history.history[metric_name], color='blue', label=metric_name)
    plt.plot(history.history['val_' + metric_name], color='green', label='val_' + metric_name)
    plt.show()

## Prepare the Data

We download the dataset and format it for training.

In [3]:
# Specify data URI
URI = './data/ENB2012_data.xlsx'

# Use pandas excel reader
df = pd.read_excel(URI)
df = df.sample(frac=1).reset_index(drop=True)

# Split the data into train and test with 80 train / 20 test
train, test = train_test_split(df, test_size=0.2)
train_stats = train.describe()

# Get Y1 and Y2 as the 2 outputs and format them as np arrays
train_stats.pop('Y1')
train_stats.pop('Y2')
train_stats = train_stats.transpose()
train_Y = format_output(train)
test_Y = format_output(test)

# Normalize the training and test data
norm_train_X = norm(train)
norm_test_X = norm(test)

## Build the Model

Here is how we'll build the model using the functional syntax. Notice that we can specify a list of outputs (i.e. `[y1_output, y2_output]`) when we instantiate the `Model()` class.

In [4]:
# Define model layers.
input_layer = Input(shape=(len(train .columns),))
first_dense = Dense(units='128', activation='relu')(input_layer)
second_dense = Dense(units='128', activation='relu')(first_dense)

# Y1 output will be fed directly from the second dense
y1_output = Dense(units='1', name='y1_output')(second_dense)
third_dense = Dense(units='64', activation='relu')(second_dense)

# Y2 output will come via the third dense
y2_output = Dense(units='1', name='y2_output')(third_dense)

# Define the model with the input layer and a list of output layers
model = Model(inputs=input_layer, outputs=[y1_output, y2_output])

print(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 128)          1152        input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 128)          16512       dense[0][0]                      
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 64)           8256        dense_1[0][0]                    
______________________________________________________________________________________________

## Configure parameters

We specify the optimizer as well as the loss and metrics for each output.

In [5]:
# Specify the optimizer, and compile the model with loss functions for both outputs
optimizer = tf.keras.optimizers.SGD(lr=0.001)
model.compile(optimizer=optimizer,
              loss={'y1_output': 'mse', 'y2_output': 'mse'},
              metrics={'y1_output': tf.keras.metrics.RootMeanSquaredError(),
                       'y2_output': tf.keras.metrics.RootMeanSquaredError()})

## Train the Model

In [6]:
# Train the model for 500 epochs
history = model.fit(norm_train_X, train_Y,
                    epochs=500, batch_size=10, validation_data=(norm_test_X, test_Y))

Train on 614 samples, validate on 154 samples
Epoch 1/500
614/614 [==============================] - 1s 887us/sample - loss: 247.1286 - y1_output_loss: 119.5041 - y2_output_loss: 125.5273 - y1_output_root_mean_squared_error: 10.9775 - y2_output_root_mean_squared_error: 11.2527 - val_loss: 54.6021 - val_y1_output_loss: 20.0642 - val_y2_output_loss: 32.9836 - val_y1_output_root_mean_squared_error: 4.5444 - val_y2_output_root_mean_squared_error: 5.8267
Epoch 2/500
614/614 [==============================] - 0s 140us/sample - loss: 30.3784 - y1_output_loss: 12.6718 - y2_output_loss: 18.2080 - y1_output_root_mean_squared_error: 3.5365 - y2_output_root_mean_squared_error: 4.2275 - val_loss: 210.1870 - val_y1_output_loss: 32.8667 - val_y2_output_loss: 171.6330 - val_y1_output_root_mean_squared_error: 5.8275 - val_y2_output_root_mean_squared_error: 13.2750
Epoch 3/500
614/614 [==============================] - 0s 137us/sample - loss: 35.9834 - y1_output_loss: 11.8537 - y2_output_loss: 23.9478 -

614/614 [==============================] - 0s 130us/sample - loss: 8.6648 - y1_output_loss: 2.8169 - y2_output_loss: 5.9331 - y1_output_root_mean_squared_error: 1.6778 - y2_output_root_mean_squared_error: 2.4186 - val_loss: 20.1813 - val_y1_output_loss: 8.1502 - val_y2_output_loss: 11.5247 - val_y1_output_root_mean_squared_error: 2.8880 - val_y2_output_root_mean_squared_error: 3.4411
Epoch 42/500
614/614 [==============================] - 0s 133us/sample - loss: 13.3060 - y1_output_loss: 3.8554 - y2_output_loss: 9.3363 - y1_output_root_mean_squared_error: 1.9718 - y2_output_root_mean_squared_error: 3.0689 - val_loss: 8.1259 - val_y1_output_loss: 2.9114 - val_y2_output_loss: 4.9776 - val_y1_output_root_mean_squared_error: 1.7281 - val_y2_output_root_mean_squared_error: 2.2671
Epoch 43/500
614/614 [==============================] - 0s 133us/sample - loss: 8.1361 - y1_output_loss: 2.6716 - y2_output_loss: 5.3984 - y1_output_root_mean_squared_error: 1.6390 - y2_output_root_mean_squared_err

Epoch 62/500
614/614 [==============================] - 0s 130us/sample - loss: 7.7005 - y1_output_loss: 2.2526 - y2_output_loss: 5.3881 - y1_output_root_mean_squared_error: 1.5073 - y2_output_root_mean_squared_error: 2.3299 - val_loss: 4.5466 - val_y1_output_loss: 1.4830 - val_y2_output_loss: 2.9076 - val_y1_output_root_mean_squared_error: 1.2409 - val_y2_output_root_mean_squared_error: 1.7340
Epoch 63/500
614/614 [==============================] - 0s 131us/sample - loss: 7.4785 - y1_output_loss: 2.0944 - y2_output_loss: 5.3528 - y1_output_root_mean_squared_error: 1.4530 - y2_output_root_mean_squared_error: 2.3167 - val_loss: 5.0822 - val_y1_output_loss: 1.7249 - val_y2_output_loss: 3.2756 - val_y1_output_root_mean_squared_error: 1.3117 - val_y2_output_root_mean_squared_error: 1.8335
Epoch 64/500
614/614 [==============================] - 0s 134us/sample - loss: 5.3407 - y1_output_loss: 1.5218 - y2_output_loss: 3.8678 - y1_output_root_mean_squared_error: 1.2244 - y2_output_root_mean_s

Epoch 83/500
614/614 [==============================] - 0s 134us/sample - loss: 3.6038 - y1_output_loss: 0.8570 - y2_output_loss: 2.7228 - y1_output_root_mean_squared_error: 0.9294 - y2_output_root_mean_squared_error: 1.6553 - val_loss: 3.0972 - val_y1_output_loss: 0.7655 - val_y2_output_loss: 2.2349 - val_y1_output_root_mean_squared_error: 0.8906 - val_y2_output_root_mean_squared_error: 1.5179
Epoch 84/500
614/614 [==============================] - 0s 130us/sample - loss: 4.8750 - y1_output_loss: 1.1290 - y2_output_loss: 3.7447 - y1_output_root_mean_squared_error: 1.0627 - y2_output_root_mean_squared_error: 1.9354 - val_loss: 8.1723 - val_y1_output_loss: 3.4208 - val_y2_output_loss: 4.5666 - val_y1_output_root_mean_squared_error: 1.8663 - val_y2_output_root_mean_squared_error: 2.1654
Epoch 85/500
614/614 [==============================] - 0s 125us/sample - loss: 3.9701 - y1_output_loss: 1.0608 - y2_output_loss: 3.0525 - y1_output_root_mean_squared_error: 1.0303 - y2_output_root_mean_s

Epoch 104/500
614/614 [==============================] - 0s 131us/sample - loss: 6.2817 - y1_output_loss: 1.6923 - y2_output_loss: 4.5330 - y1_output_root_mean_squared_error: 1.3070 - y2_output_root_mean_squared_error: 2.1386 - val_loss: 2.3208 - val_y1_output_loss: 0.6843 - val_y2_output_loss: 1.5689 - val_y1_output_root_mean_squared_error: 0.8415 - val_y2_output_root_mean_squared_error: 1.2699
Epoch 105/500
614/614 [==============================] - 0s 126us/sample - loss: 2.5042 - y1_output_loss: 0.5397 - y2_output_loss: 1.9481 - y1_output_root_mean_squared_error: 0.7369 - y2_output_root_mean_squared_error: 1.4004 - val_loss: 2.4737 - val_y1_output_loss: 0.5199 - val_y2_output_loss: 1.8843 - val_y1_output_root_mean_squared_error: 0.7268 - val_y2_output_root_mean_squared_error: 1.3948
Epoch 106/500
614/614 [==============================] - 0s 130us/sample - loss: 2.5442 - y1_output_loss: 0.6224 - y2_output_loss: 1.9220 - y1_output_root_mean_squared_error: 0.7898 - y2_output_root_mea

...

Epoch 309/500
614/614 [==============================] - 0s 125us/sample - loss: 0.7837 - y1_output_loss: 0.1793 - y2_output_loss: 0.6002 - y1_output_root_mean_squared_error: 0.4239 - y2_output_root_mean_squared_error: 0.7772 - val_loss: 0.9484 - val_y1_output_loss: 0.3231 - val_y2_output_loss: 0.6057 - val_y1_output_root_mean_squared_error: 0.5718 - val_y2_output_root_mean_squared_error: 0.7883
Epoch 310/500
614/614 [==============================] - 0s 125us/sample - loss: 0.4486 - y1_output_loss: 0.1319 - y2_output_loss: 0.3201 - y1_output_root_mean_squared_error: 0.3611 - y2_output_root_mean_squared_error: 0.5641 - val_loss: 1.2929 - val_y1_output_loss: 0.4662 - val_y2_output_loss: 0.7927 - val_y1_output_root_mean_squared_error: 0.6933 - val_y2_output_root_mean_squared_error: 0.9013
Epoch 311/500
614/614 [==============================] - 0s 127us/sample - loss: 0.5075 - y1_output_loss: 0.1224 - y2_output_loss: 0.4063 - y1_output_root_mean_squared_error: 0.3505 - y2_output_root_mea

Epoch 350/500
614/614 [==============================] - 0s 131us/sample - loss: 0.4305 - y1_output_loss: 0.1285 - y2_output_loss: 0.3024 - y1_output_root_mean_squared_error: 0.3577 - y2_output_root_mean_squared_error: 0.5501 - val_loss: 1.0717 - val_y1_output_loss: 0.3730 - val_y2_output_loss: 0.6641 - val_y1_output_root_mean_squared_error: 0.6218 - val_y2_output_root_mean_squared_error: 0.8277
Epoch 351/500
614/614 [==============================] - 0s 130us/sample - loss: 0.3849 - y1_output_loss: 0.1134 - y2_output_loss: 0.2706 - y1_output_root_mean_squared_error: 0.3371 - y2_output_root_mean_squared_error: 0.5209 - val_loss: 1.2310 - val_y1_output_loss: 0.4246 - val_y2_output_loss: 0.7715 - val_y1_output_root_mean_squared_error: 0.6623 - val_y2_output_root_mean_squared_error: 0.8902
Epoch 352/500
614/614 [==============================] - 0s 129us/sample - loss: 0.4256 - y1_output_loss: 0.1334 - y2_output_loss: 0.2932 - y1_output_root_mean_squared_error: 0.3630 - y2_output_root_mea

Epoch 391/500
614/614 [==============================] - 0s 122us/sample - loss: 0.4031 - y1_output_loss: 0.1181 - y2_output_loss: 0.2827 - y1_output_root_mean_squared_error: 0.3446 - y2_output_root_mean_squared_error: 0.5333 - val_loss: 0.7786 - val_y1_output_loss: 0.2400 - val_y2_output_loss: 0.5264 - val_y1_output_root_mean_squared_error: 0.4929 - val_y2_output_root_mean_squared_error: 0.7319
Epoch 392/500
614/614 [==============================] - 0s 124us/sample - loss: 1.0118 - y1_output_loss: 0.2945 - y2_output_loss: 0.7104 - y1_output_root_mean_squared_error: 0.5438 - y2_output_root_mean_squared_error: 0.8462 - val_loss: 0.8503 - val_y1_output_loss: 0.2757 - val_y2_output_loss: 0.5564 - val_y1_output_root_mean_squared_error: 0.5308 - val_y2_output_root_mean_squared_error: 0.7540
Epoch 393/500
614/614 [==============================] - 0s 123us/sample - loss: 0.5112 - y1_output_loss: 0.1316 - y2_output_loss: 0.3811 - y1_output_root_mean_squared_error: 0.3632 - y2_output_root_mea

Epoch 432/500
614/614 [==============================] - 0s 130us/sample - loss: 0.5757 - y1_output_loss: 0.1682 - y2_output_loss: 0.4075 - y1_output_root_mean_squared_error: 0.4103 - y2_output_root_mean_squared_error: 0.6382 - val_loss: 1.5303 - val_y1_output_loss: 0.2644 - val_y2_output_loss: 1.2221 - val_y1_output_root_mean_squared_error: 0.5221 - val_y2_output_root_mean_squared_error: 1.1215
Epoch 433/500
614/614 [==============================] - 0s 128us/sample - loss: 0.5879 - y1_output_loss: 0.1646 - y2_output_loss: 0.4188 - y1_output_root_mean_squared_error: 0.4074 - y2_output_root_mean_squared_error: 0.6496 - val_loss: 0.7508 - val_y1_output_loss: 0.2390 - val_y2_output_loss: 0.4928 - val_y1_output_root_mean_squared_error: 0.4967 - val_y2_output_root_mean_squared_error: 0.7100
Epoch 434/500
614/614 [==============================] - 0s 122us/sample - loss: 0.4193 - y1_output_loss: 0.1431 - y2_output_loss: 0.2783 - y1_output_root_mean_squared_error: 0.3770 - y2_output_root_mea

Epoch 473/500
614/614 [==============================] - 0s 134us/sample - loss: 0.3284 - y1_output_loss: 0.1168 - y2_output_loss: 0.2102 - y1_output_root_mean_squared_error: 0.3423 - y2_output_root_mean_squared_error: 0.4597 - val_loss: 0.7734 - val_y1_output_loss: 0.2561 - val_y2_output_loss: 0.4925 - val_y1_output_root_mean_squared_error: 0.5148 - val_y2_output_root_mean_squared_error: 0.7131
Epoch 474/500
614/614 [==============================] - 0s 126us/sample - loss: 0.3549 - y1_output_loss: 0.1172 - y2_output_loss: 0.2364 - y1_output_root_mean_squared_error: 0.3431 - y2_output_root_mean_squared_error: 0.4870 - val_loss: 0.9071 - val_y1_output_loss: 0.2182 - val_y2_output_loss: 0.6719 - val_y1_output_root_mean_squared_error: 0.4736 - val_y2_output_root_mean_squared_error: 0.8263
Epoch 475/500
614/614 [==============================] - 0s 126us/sample - loss: 0.3150 - y1_output_loss: 0.1037 - y2_output_loss: 0.2135 - y1_output_root_mean_squared_error: 0.3210 - y2_output_root_mea

## Evaluate the Model and Plot Metrics

In [ ]:
# Test the model and print loss and mse for both outputs
loss, Y1_loss, Y2_loss, Y1_rmse, Y2_rmse = model.evaluate(x=norm_test_X, y=test_Y)
print("Loss = {}, Y1_loss = {}, Y1_mse = {}, Y2_loss = {}, Y2_mse = {}".format(loss, Y1_loss, Y1_rmse, Y2_loss, Y2_rmse))

In [ ]:
# Plot the loss and mse
Y_pred = model.predict(norm_test_X)
plot_diff(test_Y[0], Y_pred[0], title='Y1')
plot_diff(test_Y[1], Y_pred[1], title='Y2')
plot_metrics(metric_name='y1_output_root_mean_squared_error', title='Y1 RMSE', ylim=6)
plot_metrics(metric_name='y2_output_root_mean_squared_error', title='Y2 RMSE', ylim=7)